<a href="https://colab.research.google.com/github/trinhxuandat12022003/AIO---262-Trinh-Xd/blob/excerciseW04/RAG_application_in_AIO_material_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Cài đặt các gói thư viện cần thiết

In [ ]:
! pip install -q transformers==4.41.2
! pip install -q bitsandbytes==0.43.1
! pip install -q accelerate==0.31.0
! pip install -q langchain==0.2.5
! pip install -q langchainhub==0.1.20
! pip install -q langchain-chroma==0.1.1
! pip install -q langchain-community==0.2.5
! pip install -q langchain_huggingface==0.0.3
! pip install -q python-dotenv==1.0.1
! pip install -q pypdf==4.2.0
! pip install -q numpy==1.25.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

# 2. Xây dựng vector database:

# A: Import thư viện cần thiết

In [ ]:
import torch

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer , AutoModelForCausalLM ,pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader,TextLoader
from langchain.chains import ConversationalRetrievalChain

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

# B: Đọc file pdf

In [ ]:
Loader = PyPDFLoader
FILE_PATH = "/content/YOLOv10_Tutorials.pdf"
loader = Loader(FILE_PATH)
documents = loader.load()

# C: Khởi tạo bộ tách văn bản (text splitter)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter ( chunk_size =1000 ,
                                                 chunk_overlap =100)
docs = text_splitter.split_documents(documents)
print("Number of sub - documents : ", len(docs))
for i in range(len(docs)):
  print(docs[i])
  print('\n')

Number of sub - documents :  33
page_content='AI VIET NAM – AI COURSE 2024
Tutorial: Phát hiện đối tượng trong ảnh với
YOLOv10
Dinh-Thang Duong, Nguyen-Thuan Duong, Minh-Duc Bui và
Quang-Vinh Dinh
Ngày 20 tháng 6 năm 2024
I. Giới thiệu
Object Detection (Tạm dịch: Phát hiện đối tượng) là một bài toán cổ điển thuộc lĩnh vực
Computer Vision. Mục tiêu của bài toán này là tự động xác định vị trí của các đối tượng trong
một tấm ảnh. Tính tới thời điểm hiện tại, đã có rất nhiều phương pháp được phát triển nhằm
giải quyết hiệu quả bài toán này. Trong đó, các phương pháp thuộc họ YOLO (You Only Look
Once) thu hút được sự chú ý rất lớn từ cộng đồng nghiên cứu bởi độ chính xác và tốc độ thực
thi mà loại mô hình này mang lại.
Hình 1: Logo của mô hình YOLO. Ảnh: link.
Thời gian vừa qua, Ao Wang và các cộng sự tại Đại học Thanh Hoa (Tsinghua University)
đã đề xuất mô hình YOLOv10 trong bài báo YOLOv10: Real-Time End-to-End Object
Detection [10]. Với những cải tiến mới, mô hình đã đạt được hiệu suất 

# D: Khởi tạo instance vectorization

In [ ]:
embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# E: Khởi tạo vector database

In [ ]:
vector_db = Chroma.from_documents(documents = docs, embedding = embedding)
retriever = vector_db.as_retriever()

In [ ]:
result = retriever.invoke("What is YOLOv10?")
print('Number of relevant documents: ', len(result))
for i in range(len(result)):
  print(i, " ")
  print(result[i])
  print('\n')

Number of relevant documents:  4
0  
page_content='AI VIETNAM (AIO2024) aivietnam.edu.vn
1! mkdir datasets
2! unzip -q "/ content / PlantDocv4 . zip" -d "/ content / datasets / PlantDocv4
/"
3!rm / content / PlantDocv4 .zip
Quan sát thư mục giải nén, có thể thấy bộ dữ liệu này đã được gán nhãn và đưa vào format
cấu trúc dữ liệu training theo yêu cầu của YOLO. Vì vậy, chúng ta sẽ không cần thực hiện
bước chuẩn bị dữ liệu ở bài này.
2.Cài đặt và import các thư viện cần thiết: Tương tự như phần trước, các bạn chạy
các đoạn code sau để cài đặt các gói thư viện để sử dụng được YOLOv10:
1!git clone https :// github .com/THU -MIG/ yolov10 .git
2%cd yolov10
3!pip install -q -r requirements .txt
4!pip install -e .
3.Khởi tạo mô hình YOLOv10: Chúng ta sẽ khởi tạo mộ hình YOLOv10 với phiên
bảnnano (n) từ trọng số đã được huấn luyện trên bộ dữ liệu COCO. Để tải trọng số
yolov10n.pt, các bạn chạy đoạn code sau:
1! wget https :// github .com/THU -MIG/ yolov10 / releases / download /v1 .1/
yolov10n .

# 3: Khởi tạo mô hình ngôn ngữ lớn

# A: Khai báo một số cài đặt cần thiết cho mô hình

In [ ]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# B: Khởi tạo mô hình và tokenizer

In [ ]:
MODEL_NAME = "lmsys/vicuna-7b-v1.5"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=nf4_config,
    low_cpu_mem_usage = True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

# C: Tích hợp tokenizer và model thành một pipeline để tiện sử dụng

In [ ]:
model_pipeline = pipeline (
   "text-generation",
    model =model ,
    tokenizer = tokenizer,
    max_new_tokens =512,
    pad_token_id = tokenizer.eos_token_id ,
    device_map ="auto"
)

llm = HuggingFacePipeline (
    pipeline = model_pipeline ,
)

#4: Chạy chương trình

In [ ]:
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
  {"context": retriever | format_docs , "question": RunnablePassthrough() }
  | prompt
  | llm
  | StrOutputParser ()
)

USER_QUESTION = "Hãy trình bày phiên bản YOLOv1"
output = rag_chain.invoke(USER_QUESTION)
answer = output.split('Answer:')[1].strip()
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Để sử dụng YOLOv10, bạn cần tải về file trọng số (file.pt) và khởi tạo mô hình bằng cách chạy đoạn code sau:
```python
from ultralytics import YOLOv10
model = YOLOv10 ("yolov10n.pt")
```
Sau đó, bạn có thể sử dụng mô hình này để dự đoán các ảnh mới bằng cách chạy đoạn code sau:
```python
import cv2

# Load the pre-trained YOLOv10 model
model = YOLOv10()

# Define the image to predict
img = cv2.imread("path/to/image")

# Make a prediction using the pre-trained model
boxes, scores, classes = model.predict(img)

# Draw the boxes around the predicted objects
for box in boxes:
    x, y, w, h = box
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Show the result
cv2.imshow("Prediction", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
```
Để tải ảnh cần dự đoán, bạn có thể sử dụng gdown để tải ảnh từ trang web và chuyển nó vào Google Colab.


# II: Xây dựng giao diện chat

# 1. Tải các gói thư viện:

In [ ]:
! pip install -q transformers==4.41.2
! pip install -q bitsandbytes==0.43.1
! pip install -q accelerate==0.31.0
! pip install -q langchain==0.2.5
! pip install -q langchainhub==0.1.20
! pip install -q langchain-chroma==0.1.1
! pip install -q langchain-community==0.2.5
! pip install -q langchain-openai==0.1.9
! pip install -q langchain_huggingface==0.0.3
! pip install -q chainlit==1.1.304
! pip install -q python-dotenv==1.0.1
! pip install -q pypdf==4.2.0
! npm install -g localtunnel
! pip install -q numpy==1.25.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 2. Import các gói thư viện cần thiết:

In [ ]:
import chainlit as cl
import torch

from chainlit.types import AskFileResponse

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains import ConversationalRetrievalChain

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader , TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

# 3. Cài đặt lại các hàm và instance ở file trước

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000,
                                              chunk_overlap =100)

embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# 4. Xây dựng hàm xử lý file input đầu vào

In [ ]:
def process_file (file:AskFileResponse) :
  if file.type == "text/plain":
    Loader = TextLoader
  elif file.type == "application/pdf":
    Loader = PyPDFLoader

  loader = Loader (file.path )
  documents = loader.load()
  docs = text_splitter.split_documents( documents )
  for i, doc in enumerate(docs):
    doc.metadata ["source"] = f"source_ {i}"
  return docs

# 5. Xây dựng hàm khởi tạo Chroma database

In [ ]:
def get_vector_db(file:AskFileResponse):
  docs = process_file(file)
  cl.user_session.set ("docs", docs)
  vector_db = Chroma.from_documents (documents =docs ,
                                     embedding = embedding)
  return vector_db

# 6. Khởi tạo mô hình ngôn ngữ lớn

In [ ]:
def get_huggingface_llm (model_name:str = "lmsys/vicuna-7b-v1.5",
                        max_new_token:int = 512):
  nf4_config = BitsAndBytesConfig(
    load_in_4bit =True,
    bnb_4bit_quant_type ="nf4",
    bnb_4bit_use_double_quant =True ,
    bnb_4bit_compute_dtype =torch.bfloat16
  )
  model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = nf4_config,
    low_cpu_mem_usage = True
  )
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  model_pipeline = pipeline (
   "text-generation",
    model =model,
    tokenizer = tokenizer,
    max_new_tokens = max_new_token,
    pad_token_id = tokenizer . eos_token_id,
    device_map ="auto"
  )

  llm = HuggingFacePipeline(
    pipeline = model_pipeline ,
  )
  return llm

LLM = get_huggingface_llm ()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

# 7. Khởi tạo welcome message

In [ ]:
welcome_message = """ Welcome to the PDF QA! To get started:
1. Upload a PDF or text file
2. Ask a question about the file
"""

# 8. Khởi tạo hàm on_chat_start

In [ ]:
@cl.on_chat_start
async def on_chat_start ():
  files = None
  while files is None :
    files = await cl.AskFileMessage(
      content = welcome_message,
      accept =["text/plain", "application/pdf"],
      max_size_mb =20,
      timeout =180,
    ).send()
  file = files [0]

  msg = cl.Message(content =f"Processing'{file.name}'...",
                  disable_feedback = True)
  await msg.send()

  vector_db = await cl.make_async(get_vector_db)(file)

  message_history = ChatMessageHistory ()
  memory = ConversationBufferMemory (
    memory_key ="chat_history",
    output_key ="answer",
    chat_memory = message_history,
    return_messages =True,
  )
  retriever = vector_db.as_retriever( search_type ="mmr",
                                      search_kwargs ={'k': 3})

  chain = ConversationalRetrievalChain.from_llm (
    llm =LLM ,
    chain_type ="stuff",
    retriever = retriever,
    memory =memory,
    return_source_documents = True
  )

  msg.content = f"'{ file . name }' processed . You can now ask questions !"
  await msg.update()

  cl.user_session.set("chain", chain)

# 9. Khởi tạo hàm on_message

In [ ]:
@cl.on_message
async def on_message(message:cl.Message):
  chain = cl.user_session.get("chain")
  cb = cl.AsyncLangchainCallbackHandler()
  res = await chain.ainvoke(message.content ,callbacks =[cb])
  answer = res["answer"]
  source_documents = res ["source_documents"]
  text_elements = []

  if source_documents:
    for source_idx, source_doc in enumerate(source_documents):
      source_name = f"source_ {source_idx}"
      text_elements.append(
        cl.Text(content=source_doc.page_content,
                name = source_name )
      )
    source_names = [text_el.name for text_el in text_elements ]

    if source_names:
      answer += f"\nSources : { ', '.join(source_names)}"
    else:
      answer += "\nNo sources found"

  await cl.Message(content=answer, elements =text_elements).send()

#10. Chạy chainlit app

In [ ]:
! chainlit run app.py --host 0.0.0.0 --port 8000 &>/content/logs.txt &

# 11.Expose localhost thành public host bằng localtunnel:

In [ ]:
import urllib
print ("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen ('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
!lt --port 8000 -- subdomain aivn - simple - rag

Password/Enpoint IP for localtunnel is: 34.127.83.121
your url is: https://olive-maps-occur.loca.lt
^C
